In [1]:
import sys
sys.path.insert(1, 'E:/Research/course research projects/data-driven software engineering/gui-element-detection/src')

In [2]:
import os
import glob
import matplotlib.pyplot as plt
import cv2
import requests
import numpy as np
import pandas as pd
from IPython.display import Image
from random import choice
import shutil

SEED = 42
np.random.seed(SEED)

from constants import VINS_MERGED_YOLO_FULL as dsPath

from constants import VINS_MERGED_YOLO_SPLIT_TEST_IMAGES as testImagePath
from constants import VINS_MERGED_YOLO_SPLIT_TRAIN_IMAGES as trainImagePath
from constants import VINS_MERGED_YOLO_SPLIT_VAL_IMAGES as valImagePath

from constants import VINS_MERGED_YOLO_SPLIT_TEST_LABELS as testLabelPath
from constants import VINS_MERGED_YOLO_SPLIT_TRAIN_LABELS as trainLabelPath
from constants import VINS_MERGED_YOLO_SPLIT_VAL_LABELS as valLabelPath

In [4]:
# !git clone https://github.com/ultralytics/yolov5

In [5]:
%cd yolov5

E:\Research\course research projects\data-driven software engineering\gui-element-detection\src\yolo\yolov5


In [6]:
# !pip install -r requirements.txt

In [29]:
from constants import TRAIN_SPLIT_RATIO, TEST_SPLIT_RATIO, VALIDATION_SPLIT_RATIO

imageCount = len(os.listdir(dsPath))//2

images = glob.glob(f"{dsPath}*.jpg")
xmls = glob.glob(f"{dsPath}*.txt")
assert len(images) == imageCount
assert len(xmls) == imageCount


trainCount = int(TRAIN_SPLIT_RATIO * imageCount)
valCount = int(VALIDATION_SPLIT_RATIO * imageCount)
testCount = int(TEST_SPLIT_RATIO * imageCount)

assert trainCount + valCount + testCount == imageCount

4550


In [26]:
def copy_files_splits(count, data_paths:list[str], labels_paths:list[str], origin, data_destination, label_destination):
    label_type = labels_paths[0].split(".")[-1]
    data_type_length = len(data_paths[0].split(".")[-1])
    for i in range(count):
        raw_data_file = choice(data_paths)
        data_file = raw_data_file.replace("\\","/").split("/")[-1]
        raw_label_file = raw_data_file[:-(data_type_length + 1)] + "." + label_type
        label_file = raw_label_file.replace("\\","/").split("/")[-1]

        shutil.copy(os.path.join(origin,data_file),os.path.join(data_destination, data_file))
        shutil.copy(os.path.join(origin,label_file),os.path.join(label_destination, label_file))

        data_paths.remove(raw_data_file)
        labels_paths.remove(raw_label_file)


In [30]:
# Split the data (Physically) - YOLOv5 Limitation
# copy_files_splits(trainCount,images,xmls,dsPath,trainImagePath,trainLabelPath)
# copy_files_splits(testCount,images,xmls,dsPath,testImagePath,testLabelPath)
# copy_files_splits(valCount,images,xmls,dsPath,valImagePath,valLabelPath)

In [32]:
assert len(images) + len(xmls) == 0 # all images were split into 3 categories

True

In [44]:
EPOCHS = 100
TRAIN = True


In [43]:
def set_res_dir():
    # Directory to store results
    res_dir_count = len(glob.glob('runs/train/*'))
    print(f"Current number of result directories: {res_dir_count}")
    if TRAIN:
        RES_DIR = f"results_{EPOCHS}_{res_dir_count+1}"
        print(RES_DIR)
    else:
        RES_DIR = f"results_{EPOCHS}_{res_dir_count}"
    return RES_DIR

In [45]:
# the image size used in Yolov5 GUI detection paper was 416 with batches of 16
RES_DIR = set_res_dir()
!python train.py --data dataset.yaml --weights yolov5s.pt --img 416 --epochs {EPOCHS} --batch-size 16 --name {RES_DIR}

Current number of result directories: 2
results_100_3


train: weights=yolov5s.pt, cfg=, data=dataset.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=results_100_3, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github:  YOLOv5 is out of date by 1 commit. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
fatal: cannot change to 'E:\Research\course': No such file or directory
YOLOv5  2023-3-11 Python-3.10.9 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmu